# Data preprocessing after scraping raw data

In [41]:
import pandas as pd
from difflib import SequenceMatcher

onepark = pd.read_csv("onepark.csv")
parclcik = pd.read_csv("parclick.csv")
public_park = pd.read_csv("public_park.csv")

parking = pd.DataFrame(
    {
    "parking_id" : [],
    "title" : [],
    "adress" : [],
    "star" : [],
    "parking_height" : [],
    "service" : [],
    "district" : []
    }
)

price_request = pd.DataFrame(
    {
    "price_request_id" : [],
    "begin_date" : [],
    "begin_hour" : [],
    "end_date" : [],
    "end_hour" : [],
    "duration" : [],
    "price" : [],
    "district" : [],
    "website_id" : [],
    "parking_id" : []
    }
)

onepark['title'] = onepark['title'].apply(lambda x : x.replace("Parking ", "").replace(" (Couvert)", "").replace("Public ", "").upper())
parclcik["title"] = parclcik["title"].str.upper()
public_park["public_park_id"] = [i for i in range(1,len(public_park)+1)]

public_park.to_csv("public_park.csv", index=False)

public_park

,zone_id,district,duration,price,public_park_id
0,1,1,1,4.0,1
1,1,1,2,8.0,2
2,1,1,3,16.0,3
3,1,1,4,26.0,4
4,1,1,5,38.0,5
...,...,...,...,...,...
115,2,20,2,4.8,116
116,2,20,3,9.6,117
117,2,20,4,16.8,118
118,2,20,5,25.4,119


# Creating parking entity with onepark data

In [33]:
for row_onepark in onepark.itertuples():
    is_exist = False
    
    if len(parking) > 0:
        for row_parking in parking.itertuples():
            if SequenceMatcher(None, row_onepark.title, row_parking.title).ratio() >= 0.90 :
                is_exist = True
                break
                
    if is_exist == False:
        tmp = {
            "parking_id" : "",
            "title" : row_onepark.title,
            "adress" : row_onepark.adress,
            "star" : "",
            "parking_height" : "",
            "service" : "",
            "district" : row_onepark.district
        }
        
        parking = parking.append(tmp, ignore_index = True)

parking

,parking_id,title,adress,star,parking_height,service,district
0,,SAEMES PYRAMIDES,15 Rue des Pyramides.75001 Paris,,,,1.0
1,,IMMEUBLE 223 RUE SAINT HONORÉ,223 Rue Saint Honoré.75001 Paris,,,,1.0
2,,SAEMES MEYERBEER - OPÉRA,3 rue de la Chaussée dAntin .75009 Paris,,,,1.0
3,,INDIGO LOUVRE SAMARITAINE,1 Place du Louvre .75001 Paris,,,,1.0
4,,SAEMES MADELEINE TRONCHET,21 place de la Madeleine.75008 Paris,,,,1.0
...,...,...,...,...,...,...,...
154,,IMMEUBLE SAEMES FLANDRIN (EXTÉRIEUR),face au 2 boulevard Flandrin.75016 Paris,,,,16.0
155,,SAEMES PORTE DAUTEUIL,1-3 avenue du général sarrail.75016 Paris,,,,16.0
156,,IMMEUBLE RUE MARGUERITE LONG,24 rue Marguerite Long.75017 Paris,,,,17.0
157,,TRANSDEV PARK PALAIS DE JUSTICE,52 boulevard Berthier.75017 Paris,,,,17.0


# Updating parking entity with parclick data

In [34]:
parking_tmp = parking 

for row_parclcik in parclcik.itertuples():
    is_exist = False
    
    if len(parking_tmp) > 0:
        for row_parking in parking_tmp.itertuples():
            if SequenceMatcher(None, row_parclcik.title, row_parking.title).ratio() >= 0.90 :
                is_exist = True
                
                parking_tmp.at[row_parking.Index, "star"] = row_parclcik.star
                parking_tmp.at[row_parking.Index, "parking_height"] = row_parclcik.park_height
                parking_tmp.at[row_parking.Index, "service"] = row_parclcik.service
                
                break
                
    if is_exist == False:
        tmp = {
            "parking_id" : "",
            "title" : row_parclcik.title,
            "adress" : "",
            "star" : row_parclcik.star,
            "parking_height" : row_parclcik.park_height,
            "service" : row_parclcik.service,
            "district" : row_parclcik.district
        }
        
        parking_tmp = parking_tmp.append(tmp, ignore_index = True)

parking_tmp

,parking_id,title,adress,star,parking_height,service,district
0,,SAEMES PYRAMIDES,15 Rue des Pyramides.75001 Paris,4,Couvert (1.9m),3,1.0
1,,IMMEUBLE 223 RUE SAINT HONORÉ,223 Rue Saint Honoré.75001 Paris,,,,1.0
2,,SAEMES MEYERBEER - OPÉRA,3 rue de la Chaussée dAntin .75009 Paris,,,,1.0
3,,INDIGO LOUVRE SAMARITAINE,1 Place du Louvre .75001 Paris,4,Couvert (1.9m),4,1.0
4,,SAEMES MADELEINE TRONCHET,21 place de la Madeleine.75008 Paris,0,Couvert (1.9m),3,1.0
...,...,...,...,...,...,...,...
277,,PARKING 2000,,4,Couvert (1.9m),3,18.0
278,,INDIGO - LE PARKS,,0,Couvert (1.9m),4,18.0
279,,GARAGE SAINT-GEORGES - 19E ARR.,,4,Couvert (1.9m),2,19.0
280,,INDIGO CITÉ DES SCIENCES ET DE LINDUSTRIE,,0,Couvert (1.8m),3,19.0


# Output parking entity into csv file

In [35]:
parking_tmp["parking_id"] = [i for i in range(1,len(parking_tmp)+1)]
parking_tmp['district'] = parking_tmp['district'].astype(int)

parking_tmp.to_csv("parking.csv", index=False)


# Creating price_request with onepark data

In [42]:
price_request_tmp = price_request

i = 1
for row_onepark in onepark.itertuples():
    
    if row_onepark.price >= 0 :        
        tmp_parking = None
        
        for row_parking in parking_tmp.itertuples():
            if SequenceMatcher(None, row_onepark.title, row_parking.title).ratio() >= 0.90 :
                tmp_parking = row_parking
                break

        tmp = {
            "price_request_id" : int(i),
            "begin_date" : row_onepark.begin_date,
            "begin_hour" : row_onepark.begin_hour,
            "end_date" : row_onepark.end_date,
            "end_hour" : row_onepark.end_hour,
            "duration" : row_onepark.duration,
            "price" : row_onepark.price,
            "district" : row_onepark.district,
            "website_id" : 1,
            "parking_id" : tmp_parking.parking_id
        }

        price_request_tmp = price_request_tmp.append(tmp, ignore_index = True)
        i = i+1

price_request_tmp

,price_request_id,begin_date,begin_hour,end_date,end_hour,duration,price,district,website_id,parking_id
0,1.0,2021-01-12,9.0,2021-01-12,10.0,1.0,4.3,1.0,1.0,1.0
1,2.0,2021-01-12,9.0,2021-01-12,10.0,1.0,3.0,1.0,1.0,2.0
2,3.0,2021-01-12,9.0,2021-01-12,10.0,1.0,4.0,1.0,1.0,3.0
3,4.0,2021-01-12,9.0,2021-01-12,10.0,1.0,8.8,1.0,1.0,4.0
4,5.0,2021-01-12,9.0,2021-01-12,10.0,1.0,4.3,1.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...
2997,2998.0,2021-01-12,9.0,2021-01-12,14.0,5.0,2.5,20.0,1.0,44.0
2998,2999.0,2021-01-12,9.0,2021-01-12,14.0,5.0,3.0,20.0,1.0,99.0
2999,3000.0,2021-01-12,9.0,2021-01-12,14.0,5.0,2.0,20.0,1.0,45.0
3000,3001.0,2021-01-12,9.0,2021-01-12,14.0,5.0,12.0,20.0,1.0,48.0


# Updating price_request with parclick data

In [43]:
price_request_parclick = price_request_tmp

i = len(price_request_tmp)+1

for row_parclcik in parclcik.itertuples():
    
    if row_parclcik.price >= 0 :        
        tmp_parking = None
        
        for row_parking in parking_tmp.itertuples():
            if SequenceMatcher(None, row_parclcik.title, row_parking.title).ratio() >= 0.90 :
                tmp_parking = row_parking
                break

        tmp = {
            "price_request_id" : int(i),
            "begin_date" : row_parclcik.begin_date,
            "begin_hour" : row_parclcik.begin_hour,
            "end_date" : row_parclcik.end_date,
            "end_hour" : row_parclcik.end_hour,
            "duration" : row_parclcik.duration,
            "price" : row_parclcik.price,
            "district" : row_onepark.district,
            "website_id" : 2,
            "parking_id" : tmp_parking.parking_id
        }

        price_request_parclick = price_request_parclick.append(tmp, ignore_index = True)
        i = i+1

price_request_parclick

,price_request_id,begin_date,begin_hour,end_date,end_hour,duration,price,district,website_id,parking_id
0,1.0,2021-01-12,9.0,2021-01-12,10.0,1.0,4.3,1.0,1.0,1.0
1,2.0,2021-01-12,9.0,2021-01-12,10.0,1.0,3.0,1.0,1.0,2.0
2,3.0,2021-01-12,9.0,2021-01-12,10.0,1.0,4.0,1.0,1.0,3.0
3,4.0,2021-01-12,9.0,2021-01-12,10.0,1.0,8.8,1.0,1.0,4.0
4,5.0,2021-01-12,9.0,2021-01-12,10.0,1.0,4.3,1.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...
4647,4648.0,2021-01-12,9.0,2021-01-12,14.0,5.0,23.1,20.0,2.0,235.0
4648,4649.0,2021-01-12,9.0,2021-01-12,14.0,5.0,28.8,20.0,2.0,236.0
4649,4650.0,2021-01-12,9.0,2021-01-12,14.0,5.0,14.0,20.0,2.0,188.0
4650,4651.0,2021-01-12,9.0,2021-01-12,14.0,5.0,18.2,20.0,2.0,198.0


# Output price_request in csv file

In [44]:
price_request_parclick['price_request_id'] = price_request_parclick['price_request_id'].astype(int)
price_request_parclick['begin_hour'] = price_request_parclick['begin_hour'].astype(int)
price_request_parclick['end_hour'] = price_request_parclick['end_hour'].astype(int)
price_request_parclick['duration'] = price_request_parclick['duration'].astype(int)
price_request_parclick['website_id'] = price_request_parclick['website_id'].astype(int)
price_request_parclick['district'] = price_request_parclick['district'].astype(int)
price_request_parclick['parking_id'] = price_request_parclick['parking_id'].astype(int)

price_request_parclick.to_csv("price_request.csv", index=False)